Trying to analyze the gt points' distribution with respect to predicted points location

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
%matplotlib qt

import scipy.stats

import torchvision.datasets as dsets
import torchvision.transforms as transforms
# from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader
from torch.autograd import Variable

import my_model
from utilities import MyTrainDataSet, MyTestDataSet, load_data_2, load_test_data, min_max_scaling, min_max_scaling_radius, normalize_one, construct_train_valid_tensor, construct_test_tensor, show_statistic

# get training data

In [2]:
input_train = np.load("input_tensor.npy")
gt_label = np.load("gt_label_tensor.npy")
pd_label = np.load("pd_label_tensor.npy")

input_valid_train = np.load("input_valid_tensor.npy")
gt_valid_label = np.load("gt_valid_label_tensor.npy")
pd_valid_label = np.load("pd_valid_label_tensor.npy")

In [3]:
input_tensor = torch.Tensor(input_train)
gt_label_tensor = torch.Tensor(gt_label)
pd_label_tensor = torch.Tensor(pd_label)

input_valid_tensor = torch.Tensor(input_valid_train)
gt_valid_label_tensor = torch.Tensor(gt_valid_label)
pd_valid_label_tensor = torch.Tensor(pd_valid_label)

In [4]:
class MDN(nn.Module):
    def __init__(self, n_input, n_hidden, n_gaussians):
        super(MDN, self).__init__()
        self.l_h = nn.Sequential(
            nn.Linear(n_input, n_hidden),
            nn.Tanh()
        )
        self.l_pi = nn.Linear(n_hidden, n_gaussians)
        
        self.l_mu_x = nn.Linear(n_hidden, n_gaussians)
        self.l_sigma_x = nn.Linear(n_hidden, n_gaussians)
        
        self.l_mu_y = nn.Linear(n_hidden, n_gaussians)
        self.l_sigma_y = nn.Linear(n_hidden, n_gaussians)
        
        self.l_mu_z = nn.Linear(n_hidden, n_gaussians)
        self.l_sigma_z = nn.Linear(n_hidden, n_gaussians)
        
        '''
        self.l_correlation_x_y = nn.Sequential(
            nn.Linear(n_hidden, n_gaussians),
            nn.Tanh()
        )
        self.l_correlation_x_z = nn.Sequential(
            nn.Linear(n_hidden, n_gaussians),
            nn.Tanh()
        )
        self.l_correlation_y_z = nn.Sequential(
            nn.Linear(n_hidden, n_gaussians),
            nn.Tanh()
        )
        '''
    def forward(self, x):
        h = self.l_h(x)
        # print("h", h.shape)
        # print("h[0]", h[0, :])
        pi = F.softmax(self.l_pi(h), -1)
        # print("pi", pi.shape)
        # print("pi[0]", pi[0, :])
        mu_x = self.l_mu_x(h)
        # print("mu_x", pi.shape)
        mu_y = self.l_mu_y(h)
        # print("mu_y", pi.shape)
        mu_z = self.l_mu_z(h)
        # print("mu_z", pi.shape)
        
        # use exp to ensure positive range
        sigma_x = torch.exp(self.l_sigma_x(h))
        # print("sigma_x", sigma_x.shape)
        sigma_y = torch.exp(self.l_sigma_y(h))
        # print("sigma_y", sigma_y.shape)
        sigma_z = torch.exp(self.l_sigma_z(h))
        # print("sigma_z", sigma_z.shape)
        '''
        # use tanh to ensoure range of (-1, 1)
        correlation_x_y = self.l_correlation_x_y(h)
        # print("correlation_x_y", pi.shape)
        # print("correlation_x_y[0]", correlation_x_y[0, :])
        correlation_x_z = self.l_correlation_x_z(h)
        # print("correlation_x_z", pi.shape)
        # print("correlation_x_z[0]", correlation_x_z[0, :])
        correlation_y_z = self.l_correlation_y_z(h)
        # print("correlation_y_z", pi.shape)
        # print("correlation_y_z[0]", correlation_y_z[0, :])
        '''
        return pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z


In [5]:
def mdn_loss_fn(y, pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z):
    # print("mu_x shape", mu_x.shape)
    # mu = torch.stack((mu_x, mu_y, mu_z), 2)
    # print("mu shape ", mu.shape)
    # print("mu[0]", mu[0])
    
    ##############################################################################
    # print("sigma x,y,z size", sigma_x.shape, sigma_y.shape, sigma_z.shape)
    
    size = y.shape[0]
    n_gaussians = pi.shape[1]
    # print("sample size: ", size)
    # print("num of gaus: ", n_gaussians)
    # print("correlation size: ", correlation_x_y.shape)
    # build mean matrix
    mean = torch.stack((mu_x, mu_y, mu_z), dim=2)
    # build covariance matrix with standard trivariate normal distribution
    cov = torch.zeros([size, n_gaussians, 3, 3])
    cov[:, :, 0, 0] = sigma_x**2
    cov[:, :, 1, 1] = sigma_y**2
    cov[:, :, 2, 2] = sigma_z**2
    
    # print("mean: ", mean.shape)
    '''
    print("cov: ", cov.shape)
    print("sigma x", (sigma_x**2)[0, 0])
    print("sigma y", (sigma_y**2)[0, 0])
    print("sigma z", (sigma_z**2)[0, 0])
    print("cov", cov[0, 0])
    print("sigma x", (sigma_x**2)[0, 1])
    print("sigma y", (sigma_y**2)[0, 1])
    print("sigma z", (sigma_z**2)[0, 1])
    print("cov", cov[0, 1])
    '''
    # print("mean is cuda: ", mean.is_cuda)
    # print(" cov is cuda: ", cov.is_cuda)
    
    
    
    
    '''
    w = y[0]**2*(correlation_y_z**2 - 1) +\
        y[1]**2*(correlation_x_z**2 - 1) +\
        y[2]**2*(correlation_x_y**2 - 1) +\
        2*(y[0]*y[:1]*(correlation_x_y - correlation_x_z*correlation_y_z) +\
           y[0]*y[:2]*(correlation_x_z - correlation_x_y*correlation_y_z) +\
           y[1]*y[:2]*(correlation_y_z - correlation_x_y*correlation_x_z))
    
    return w
    '''
    '''
    cov = torch.Tensor([[sigma_x**2, 0, 0], [0, sigma_y**2, 0], [0, 0, sigma_z**2]])
    cov[1, 0] = correlation_x_y*sigma_x*sigma_y
    cov[2, 0] = correlation_x_z*sigma_x*sigma_z
    cov[2, 1] = correlation_y_z*sigma_y*sigma_z
    cov[0, 1] = correlation_x_y*sigma_x*sigma_y
    cov[0, 2] = correlation_x_z*sigma_x*sigma_z
    cov[1, 2] = correlation_y_z*sigma_y*sigma_z
    '''
    
    # mean: N x n_gaussian x 3
    # cov: N x 5 x 3 x 3
    m = torch.distributions.multivariate_normal.MultivariateNormal(loc=mean, covariance_matrix=cov)
    # expand y to N x n_gaussian x 3
    y = y.unsqueeze(1).repeat(1, n_gaussians, 1)
    
    # print("y size", y.shape) # 14985 x 5 x 3
    
    # y: [x, y, z]; row: x, y, z; column: N, number of samples
    # y: N x 3
    # loss: N x 1
    # print(y_normal.is_cuda)
    likelihood = torch.exp(m.log_prob(y))
    
    # print("likelihood shape: ", likelihood.shape) # 14985 x 5
    # print("pi shape: ", pi.shape) # 14985 x 5
    loss = torch.sum(likelihood * pi, dim=1) # 14985 X 1
    '''
    for i in range(loss.shape[0]):
        if loss[i] > 1.5:
            print("loss: ", loss[i])
            print("likelihood: ", likelihood[i])
            print("mu array  : ", pi[i])
            # print("likelihood shape: ", likelihood.shape) # 14985 x 5
    '''
    loss = -torch.log(loss)
    return torch.mean(loss)

In [6]:
n_gaussian = 2

In [7]:
model_mdn = MDN(3, n_hidden=20, n_gaussians=n_gaussian)
#if torch.cuda.is_available():
#    model.cuda()

optimizer = torch.optim.Adam(model_mdn.parameters())

In [ ]:
loss_list = []
for epoch in range(1000):
    optimizer.zero_grad()
    pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z = model_mdn(pd_label_tensor)
    loss = mdn_loss_fn(gt_label_tensor, pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z)
    loss.backward()
    optimizer.step()
    loss_list.append(loss.data.item())
    
    total_valid_loss = 0.0
    valid_pred = []
    pi_valid, mu_x_valid, mu_y_valid, mu_z_valid, sigma_x_valid, sigma_y_valid, sigma_z_valid = model_mdn(pd_valid_label_tensor)
    loss = mdn_loss_fn(gt_valid_label_tensor, pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z)
    
    if epoch % 10 == 0:
        print("-------epoch ", epoch, "-------")
        print(loss.data.tolist())

In [ ]:
pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z = model_mdn(pd_label_tensor)

size = pi.shape[0]
n_gaussians = pi.shape[1]
# print("sample size: ", size)
# print("num of gaus: ", n_gaussians)
# print("correlation size: ", correlation_x_y.shape)
# build mean matrix
mean = torch.stack((mu_x, mu_y, mu_z), dim=2)
# build covariance matrix with standard trivariate normal distribution
cov = torch.zeros([size, n_gaussians, 3, 3])
cov[:, :, 0, 0] = sigma_x**2
cov[:, :, 1, 1] = sigma_y**2
cov[:, :, 2, 2] = sigma_z**2

m = torch.distributions.multivariate_normal.MultivariateNormal(loc=mean, covariance_matrix=cov)


In [ ]:
pi.shape

In [ ]:
pd = m.rsample()

In [ ]:
k = torch.multinomial(pi, 1).view(-1)
PPD = torch.zeros([size, 3])
for i in range(pd.shape[0]):
    PPD[i, :] = pd[i,k[i],:]

In [ ]:
PPD.shape

In [ ]:
PPD = PPD.detach().numpy()

In [ ]:
mu_x = mu_x.detach().numpy()
mu_y = mu_y.detach().numpy()
mu_z = mu_z.detach().numpy()

In [ ]:
plt.plot(gt_label[:,0], label='gt', linestyle='--')
plt.plot(pd_label[:,0], label='pd', linestyle='--')
plt.plot(PPD[:,0], label='pd_mdn', linestyle='--')
plt.plot(mu_x[:,0], label='mu 1')
plt.plot(mu_x[:,1], label='mu 2')
# plt.plot(mu_x[:,2], label='mu 3')
# plt.plot(mu_x[:,3], label='mu 4')
# plt.plot(mu_x[:,4], label='mu 5')
plt.legend()

In [ ]:
gt_label.shape

In [ ]:
index = 0
radius = 0.2
density = 40 # sample size alone radius
x_center, y_center, z_center = pd_label[index]

s_x = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
s_y = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
s_z = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])

x_min = x_center - radius
y_min = y_center - radius
z_min = z_center - radius
unit_size = radius/density

for i in range(density*2 + 1):
    for j in range(density*2 + 1):
        for k in range(density*2 + 1):
            s_x[i, j, k] = x_min + i*unit_size
            s_y[i, j, k] = y_min + j*unit_size
            s_z[i, j, k] = z_min + k*unit_size
            


pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z = model_mdn(pd_label_tensor[index])
pi = pi.unsqueeze(0)
mu_x = mu_x.unsqueeze(0)
mu_y = mu_y.unsqueeze(0)
mu_z = mu_z.unsqueeze(0)
sigma_x = sigma_x.unsqueeze(0)
sigma_y = sigma_y.unsqueeze(0)
sigma_z = sigma_z.unsqueeze(0)
# print(pi.shape)
# print(mu_x.shape)
# print(mu_y.shape)
# print(mu_z.shape)
# print(sigma_x.shape)
# print(sigma_y.shape)
# print(sigma_z.shape)

size = pi.shape[0]
n_gaussians = pi.shape[1]
# build mean matrix
mean = torch.stack((mu_x, mu_y, mu_z), dim=2)
# print(mean.shape)
# build covariance matrix with standard trivariate normal distribution
cov = torch.zeros([size, n_gaussians, 3, 3])
cov[:, :, 0, 0] = sigma_x**2
cov[:, :, 1, 1] = sigma_y**2
cov[:, :, 2, 2] = sigma_z**2
# print(cov.shape)

# mean[0, 0] = -0.18
# mean[0, 1] = 0
# mean[0, 2] = -0.01

normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(loc=mean, covariance_matrix=cov)

pdf = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
for i in range(density*2 + 1):
    print(i)
    for j in range(density*2 + 1):
        for k in range(density*2 + 1):
            likelihood = torch.exp(normal_3d.log_prob(torch.tensor([[s_x[i, j, k], s_y[i, j, k], s_z[i, j, k]]])))
            # get mixture sum
            # pdf[i, j, k] = torch.sum(likelihood * pi, dim=1)
            pdf[i, j, k] = likelihood[0, 0]
        


In [ ]:
index = 0
radius = 0.2
density = 40 # sample size alone radius
x_center, y_center, z_center = pd_label[index]

s_x = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
s_y = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
s_z = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])

x_min = x_center - radius
y_min = y_center - radius
z_min = z_center - radius
unit_size = radius/density

for i in range(density*2 + 1):
    for j in range(density*2 + 1):
        for k in range(density*2 + 1):
            s_x[i, j, k] = x_min + i*unit_size
            s_y[i, j, k] = y_min + j*unit_size
            s_z[i, j, k] = z_min + k*unit_size
            


pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z = model_mdn(pd_label_tensor[index])
pi = pi.unsqueeze(0)
mu_x = mu_x.unsqueeze(0)
mu_y = mu_y.unsqueeze(0)
mu_z = mu_z.unsqueeze(0)
sigma_x = sigma_x.unsqueeze(0)
sigma_y = sigma_y.unsqueeze(0)
sigma_z = sigma_z.unsqueeze(0)
# print(pi.shape)
# print(mu_x.shape)
# print(mu_y.shape)
# print(mu_z.shape)
# print(sigma_x.shape)
# print(sigma_y.shape)
# print(sigma_z.shape)

size = pi.shape[0]
n_gaussians = pi.shape[1]
# build mean matrix
mean = torch.stack((mu_x, mu_y, mu_z), dim=2)
# print(mean.shape)
# build covariance matrix with standard trivariate normal distribution
cov = torch.zeros([size, n_gaussians, 3, 3])
cov[:, :, 0, 0] = sigma_x**2
cov[:, :, 1, 1] = sigma_y**2
cov[:, :, 2, 2] = sigma_z**2
# print(cov.shape)

# mean[0, 0] = 0.01
# mean[0, 1] = 0
# mean[0, 2] = -0.01

normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(loc=mean, covariance_matrix=cov)

pdf_1 = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
for i in range(density*2 + 1):
    print(i)
    for j in range(density*2 + 1):
        for k in range(density*2 + 1):
            likelihood = torch.exp(normal_3d.log_prob(torch.tensor([[s_x[i, j, k], s_y[i, j, k], s_z[i, j, k]]])))
            # get mixture sum
            # pdf_![i, j, k] = torch.sum(likelihood * pi, dim=1)
            pdf_1[i, j, k] = likelihood[0, 1]



In [ ]:
index = 0
radius = 0.2
density = 40 # sample size alone radius
# radius = 0.01
# density = 20 # sample size alone radius
x_center, y_center, z_center = pd_label[index]

s_x = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
s_y = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
s_z = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])

x_min = x_center - radius
y_min = y_center - radius
z_min = z_center - radius
unit_size = radius/density

for i in range(density*2 + 1):
    for j in range(density*2 + 1):
        for k in range(density*2 + 1):
            s_x[i, j, k] = x_min + i*unit_size
            s_y[i, j, k] = y_min + j*unit_size
            s_z[i, j, k] = z_min + k*unit_size
            


pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z = model_mdn(pd_label_tensor[index])
pi = pi.unsqueeze(0)
mu_x = mu_x.unsqueeze(0)
mu_y = mu_y.unsqueeze(0)
mu_z = mu_z.unsqueeze(0)
sigma_x = sigma_x.unsqueeze(0)
sigma_y = sigma_y.unsqueeze(0)
sigma_z = sigma_z.unsqueeze(0)
# print(pi.shape)
# print(mu_x.shape)
# print(mu_y.shape)
# print(mu_z.shape)
# print(sigma_x.shape)
# print(sigma_y.shape)dd
# print(sigma_z.shape)

size = pi.shape[0]
n_gaussians = pi.shape[1]
# build mean matrix
mean = torch.stack((mu_x, mu_y, mu_z), dim=2)
# print(mean.shape)
# build covariance matrix with standard trivariate normal distribution
cov = torch.zeros([size, n_gaussians, 3, 3])
cov[:, :, 0, 0] = sigma_x**2
cov[:, :, 1, 1] = sigma_y**2
cov[:, :, 2, 2] = sigma_z**2
# print(cov.shape)

# mean[0, 0] = -0.18
# mean[0, 1] = 0
# mean[0, 2] = -0.01

normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(loc=mean, covariance_matrix=cov)

        
pdf_all = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
for i in range(density*2 + 1):
    print(i)
    for j in range(density*2 + 1):
        for k in range(density*2 + 1):
            likelihood = torch.exp(normal_3d.log_prob(torch.tensor([[s_x[i, j, k], s_y[i, j, k], s_z[i, j, k]]])))
            # get mixture sum
            pdf_all[i, j, k] = torch.sum(likelihood * pi, dim=1)
            # pdf_4[i, j, k] = likelihood[0, 4]



In [ ]:
print(pd_label_tensor[0])
print(pi[0])
print(mu_x[0])
print(mu_y[0])
print(mu_z[0])
print(sigma_x[0])
print(sigma_y[0])
print(sigma_z[0])

In [ ]:
import numpy as np
from mayavi import mlab

src = mlab.pipeline.scalar_field(pdf)
mlab.pipeline.iso_surface(src, contours=[0.1, ], opacity=0.3, color = (0.8,0.0,0.0))
# mlab.pipeline.iso_surface(src, contours=[pdf.max()-0.1*pdf.ptp(), ],)

src_1 = mlab.pipeline.scalar_field(pdf_1)
mlab.pipeline.iso_surface(src_1, contours=[0.1, ], opacity=0.3, color = (0.8,0.0,0.0))
# mlab.pipeline.iso_surface(src_1, contours=[pdf_1.max()-0.1*pdf_1.ptp(), ],)

# src_half = mlab.pipeline.scalar_field(pdf_half)
# mlab.pipeline.iso_surface(src_half, contours=[0.1, ], opacity=0.3, color = (0.0,0.8,0.0))
# mlab.pipeline.iso_surface(src, contours=[pdf.max()-0.1*pdf.ptp(), ],)

# src_1_half = mlab.pipeline.scalar_field(pdf_1_half)
# mlab.pipeline.iso_surface(src_1_half, contours=[0.1, ], opacity=0.3, color = (0.0,0.8,0.0))
# mlab.pipeline.iso_surface(src_1, contours=[pdf_1.max()-0.1*pdf_1.ptp(), ],)

src_all = mlab.pipeline.scalar_field(pdf_all)
mlab.pipeline.iso_surface(src_all, contours=[0.1, ], opacity=0.3, color = (0.0,0.0,0.8))
# mlab.pipeline.iso_surface(src_1, contours=[pdf_1.max()-0.1*pdf_1.ptp(), ],)

In [ ]:
8.1318e-02 + 3.6224e-01 + 9.2028e-07 + 1.2253e-02 + 1.7447e-06

In [ ]:
pi

In [ ]:
sigma_x

In [ ]:
sigma_y

In [ ]:
sigma_z

In [ ]:
mu_x

In [ ]:
pi.unsqueeze(0).shape

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt



fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')




ax.scatter(s_x.flatten(), s_y.flatten(), s_z.flatten(), c='r', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

In [ ]:
pd_label[0]

In [ ]:
a

In [ ]:
b

In [ ]:
c

In [ ]:
pd_label[0]

In [ ]:

for i in range(21):
    for j in range(21):
        for k in range(21):
            s[i, j, k] = torch.exp(normal_3d.log_prob(torch.FloatTensor([i-10, j-10, k-10])))
            
src = mlab.pipeline.scalar_field(s)
mlab.pipeline.iso_surface(src, contours=[s.min()+0.1*s.ptp(), ], opacity=0.3)
mlab.pipeline.iso_surface(src, contours=[s.max()-0.1*s.ptp(), ],)

In [ ]:
    m = torch.distributions.multivariate_normal.MultivariateNormal(loc=mean, covariance_matrix=cov)

k = torch.multinomial(pi, 1).view(-1)
y_pred = torch.normal(mu, sigma)[np.arange(n_samples), k].data

In [ ]:
a = torch.tensor([0.1585, 0.1787, 0.1571, 0.3795, 0.1261])

In [ ]:
a = torch.tensor([0.0165, 0.0245, 0.1137, 2.8152, 0.0082])
b = torch.tensor([0.0339, 0.0643, 0.1532, 0.5262, 0.2225])

In [ ]:
torch.sum(a*b, dim=0)

In [ ]:
(a*b).shape

In [ ]:
a = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12]])

In [ ]:

a

In [ ]:
a[:, [1,2]]

In [ ]:
a.shape

In [ ]:
c

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
c.shape

In [ ]:
d = torch.stack((a, b), dim=1)

In [ ]:
d

In [ ]:
a.dtype

In [ ]:
d[:,:,0]

In [ ]:
d[0,0,:]

In [ ]:
F.softmax(a, -1)

In [ ]:
a

In [ ]:
math.exp(7) / (math.exp(1) + math.exp(7))

In [ ]:
F.softmax(a, 1)

In [ ]:
a.shape

In [ ]:
mean = torch.stack((mu_x, mu_y, mu_z), dim=2)

In [ ]:
w[:, :, 0, 0].shape

In [ ]:
n_gaussians = 5
size = 14000

In [ ]:
e = e.unsqueeze(0).repeat(n_gaussians, 1, 1).unsqueeze(0).repeat(size, 1, 1, 1)

In [ ]:
e.shape

In [ ]:
e = e.repeat(5, 1, 1)

In [ ]:
e.shape

In [ ]:
e = e.unsqueeze(0)

In [ ]:
e.shape

In [ ]:
e = e.repeat(10, 1, 1, 1)

In [ ]:
e.shape

In [ ]:
e[0][0]

In [ ]:
a = torch.tensor([[1,2,3],[4,5,6]])

In [ ]:
a

In [ ]:
a**2

In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
# ax.plot3D(test[:,0], test[:,1], test[:,2], 'gray')
ax.plot3D(input_tensor[0, :, 0], input_tensor[0, :, 1], input_tensor[0, :, 2], 'gray')
ax.scatter(gt_label_tensor[0,0], gt_label_tensor[0,1], gt_label_tensor[0,2], 'green')
ax.scatter(pd_label_tensor[0,0], pd_label_tensor[0,1], pd_label_tensor[0,2], 'red')
# ax.plot3D(gt_label_tensor[0:2,0], gt_label_tensor[0:2,1], gt_label_tensor[0:2,2], 'green')
# ax.plot3D(pd_label_tensor[0:2,0], pd_label_tensor[0:2,1], pd_label_tensor[0:2,2], 'red')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
# ax.plot3D(test[:,0], test[:,1], test[:,2], 'gray')
ax.plot3D(input_tensor[:3, 0, 0], input_tensor[:3, 0, 1], input_tensor[:3, 0, 2], 'gray')
ax.scatter(input_tensor[:3, 0, 0], input_tensor[:3, 0, 1], input_tensor[:3, 0, 2], c = 'gray')

ax.plot3D(input_tensor[2:6, 0, 0], input_tensor[2:6, 0, 1], input_tensor[2:6, 0, 2], 'green')
ax.scatter(input_tensor[2:6, 0, 0], input_tensor[2:6, 0, 1], input_tensor[2:6, 0, 2], c = 'green')

ax.plot3D(gt_label_tensor[0:3,0], gt_label_tensor[0:3,1], gt_label_tensor[0:3,2], 'black')
ax.plot3D(pd_label_tensor[0:3,0], pd_label_tensor[0:3,1], pd_label_tensor[0:3,2], 'red')
ax.scatter(pd_label_tensor[0:3,0], pd_label_tensor[0:3,1], pd_label_tensor[0:3,2], c = 'red')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

In [ ]:
x_pd = pd_label_tensor[:, 0].reshape(-1,1)
y_pd = pd_label_tensor[:, 1].reshape(-1,1)
z_pd = pd_label_tensor[:, 2].reshape(-1,1)

x_gt = gt_label_tensor[:, 0].reshape(-1,1)
y_gt = gt_label_tensor[:, 1].reshape(-1,1)
z_gt = gt_label_tensor[:, 2].reshape(-1,1)

In [ ]:
x_gt = x_gt.reshape(-1,1)

In [ ]:
x_gt.shape

In [ ]:
plt.plot(x_gt)
plt.plot(x_pd)

### Apply MDN on x

In [ ]:
n_samples = 14985

x_data = torch.Tensor(x_pd)
y_data = torch.Tensor(x_gt)

plt.figure(figsize=(8, 8))
plt.scatter(x_data, y_data, alpha=0.4)
plt.show()

In [ ]:
x_data.shape

In [ ]:
class MDN(nn.Module):
    def __init__(self, n_hidden, n_gaussians):
        super(MDN, self).__init__()
        self.z_h = nn.Sequential(
            nn.Linear(1, n_hidden),
            nn.Tanh()
        )
        self.z_pi = nn.Linear(n_hidden, n_gaussians)
        self.z_mu = nn.Linear(n_hidden, n_gaussians)
        self.z_sigma = nn.Linear(n_hidden, n_gaussians)
    
    def forward(self, x):
        z_h = self.z_h(x)
        pi = F.softmax(self.z_pi(z_h), -1)
        mu = self.z_mu(z_h)
        sigma = torch.exp(self.z_sigma(z_h))
        return pi, mu, sigma

model = MDN(n_hidden=20, n_gaussians=5)

optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def mdn_loss_fn(y, mu, sigma, pi):
    m = torch.distributions.Normal(loc=mu, scale=sigma)
    loss = torch.exp(m.log_prob(y))
    loss = torch.sum(loss * pi, dim=1)
    loss = -torch.log(loss)
    return torch.mean(loss)

In [ ]:
for epoch in range(10000):
    pi, mu, sigma = model(x_data)
    loss = mdn_loss_fn(y_data, mu, sigma, pi)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(loss.data.tolist())

In [ ]:
pi, mu, sigma = model(x_data)

In [ ]:
print(pi.shape)
print(mu.shape)
print(sigma.shape)

In [ ]:
# get the index of the highest index of the 5 elements in each row
k = torch.multinomial(pi, 1).view(-1)
y_pred = torch.normal(mu, sigma)[np.arange(n_samples), k].data

In [ ]:
plt.figure(figsize=(8, 8))
# lt.scatter(x_data, y_data, alpha=0.4)
# plt.scatter(x_data, y_pred, alpha=0.8, color='red')
# plt.scatter(x_test[0], y_pred[0], alpha=0.4, color='yellow')
plt.scatter(x_data, mu[:,0].detach(), label='mean 0')
plt.scatter(x_data, mu[:,1].detach(), label='mean 1')
plt.scatter(x_data, mu[:,2].detach(), label='mean 2')
plt.scatter(x_data, mu[:,3].detach(), label='mean 3')
plt.scatter(x_data, mu[:,4].detach(), label='mean 4')
# plt.plot(x_test.numpy(), mu[:,0].detach().numpy())
# plt.scatter(x_data[:20], y_data[:20], alpha=0.4)
plt.legend()
plt.show()

In [ ]:
pi = pi.detach().numpy()
mu = mu.detach().numpy()
sigma = sigma.detach().numpy()


In [ ]:
x_data = x_data.numpy()

In [ ]:
p = np.zeros((n_samples, 100))
x = np.linspace(-2, 2, 100)

for i in range(n_samples):
    each = []
    for j in range(5):
        if (j == 0):
            y = [pi[i, j] * ele for ele in scipy.stats.norm.pdf(x, mu[i, j],sigma[i, j])]
            each.append(y)
            cumulated_y = y
        else:
            y = [pi[i, j] * ele for ele in scipy.stats.norm.pdf(x, mu[i, j],sigma[i, j])]
            each.append(y)
            cumulated_y = [p + q for p, q in zip(cumulated_y, y)]
    p[i, :] = cumulated_y

In [ ]:
p.shape

In [ ]:
ax = plt.axes(projection='3d')
for i in range(0,n_samples):
    ax.plot3D([x_data[i, 0]]*100, x, p[i, :])

ax.set_xlabel('index')
ax.set_ylabel('x')
ax.set_zlabel('pdf')

In [ ]:
x_data[0, 1]

### apply MDN on 3D projection

In [ ]:
mean_1_1 = torch.Tensor([1, 1, 1])
mean_1_2 = torch.Tensor([2, 2, 2])
mean_1_3 = torch.Tensor([3, 3, 3])
mean_1_4 = torch.Tensor([4, 4, 4])
mean_1_5 = torch.Tensor([5, 5, 5])

mean_2_1 = torch.Tensor([6, 6, 6])
mean_2_2 = torch.Tensor([7, 7, 7])
mean_2_3 = torch.Tensor([8, 8, 8])
mean_2_4 = torch.Tensor([9, 9, 9])
mean_2_5 = torch.Tensor([10, 10, 10])

sigma_1 = 1
sigma_2 = 1
sigma_3 = 1
cov_1_1 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_1_1[1, 0] = rho_1_2*sigma_1*sigma_2
cov_1_1[2, 0] = rho_1_3*sigma_1*sigma_3
cov_1_1[2, 1] = rho_2_3*sigma_2*sigma_3
cov_1_1[0, 1] = rho_1_2*sigma_1*sigma_2
cov_1_1[0, 2] = rho_1_3*sigma_1*sigma_3
cov_1_1[1, 2] = rho_2_3*sigma_2*sigma_3

sigma_1 = 2
sigma_2 = 2
sigma_3 = 2
cov_1_2 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_1_2[1, 0] = rho_1_2*sigma_1*sigma_2
cov_1_2[2, 0] = rho_1_3*sigma_1*sigma_3
cov_1_2[2, 1] = rho_2_3*sigma_2*sigma_3
cov_1_2[0, 1] = rho_1_2*sigma_1*sigma_2
cov_1_2[0, 2] = rho_1_3*sigma_1*sigma_3
cov_1_2[1, 2] = rho_2_3*sigma_2*sigma_3
                         
sigma_1 = 3
sigma_2 = 3
sigma_3 = 3
cov_1_3 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_1_3[1, 0] = rho_1_2*sigma_1*sigma_2
cov_1_3[2, 0] = rho_1_3*sigma_1*sigma_3
cov_1_3[2, 1] = rho_2_3*sigma_2*sigma_3
cov_1_3[0, 1] = rho_1_2*sigma_1*sigma_2
cov_1_3[0, 2] = rho_1_3*sigma_1*sigma_3
cov_1_3[1, 2] = rho_2_3*sigma_2*sigma_3
                         
sigma_1 = 4
sigma_2 = 4
sigma_3 = 4
cov_1_4 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_1_4[1, 0] = rho_1_2*sigma_1*sigma_2
cov_1_4[2, 0] = rho_1_3*sigma_1*sigma_3
cov_1_4[2, 1] = rho_2_3*sigma_2*sigma_3
cov_1_4[0, 1] = rho_1_2*sigma_1*sigma_2
cov_1_4[0, 2] = rho_1_3*sigma_1*sigma_3
cov_1_4[1, 2] = rho_2_3*sigma_2*sigma_3
                         
sigma_1 = 5
sigma_2 = 5
sigma_3 = 5
cov_1_5 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_1_5[1, 0] = rho_1_2*sigma_1*sigma_2
cov_1_5[2, 0] = rho_1_3*sigma_1*sigma_3
cov_1_5[2, 1] = rho_2_3*sigma_2*sigma_3
cov_1_5[0, 1] = rho_1_2*sigma_1*sigma_2
cov_1_5[0, 2] = rho_1_3*sigma_1*sigma_3
cov_1_5[1, 2] = rho_2_3*sigma_2*sigma_3
                         
sigma_1 = 6
sigma_2 = 6
sigma_3 = 6
cov_2_1 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_2_1[1, 0] = rho_1_2*sigma_1*sigma_2
cov_2_1[2, 0] = rho_1_3*sigma_1*sigma_3
cov_2_1[2, 1] = rho_2_3*sigma_2*sigma_3
cov_2_1[0, 1] = rho_1_2*sigma_1*sigma_2
cov_2_1[0, 2] = rho_1_3*sigma_1*sigma_3
cov_2_1[1, 2] = rho_2_3*sigma_2*sigma_3

sigma_1 = 7
sigma_2 = 7
sigma_3 = 7
cov_2_2 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_2_2[1, 0] = rho_1_2*sigma_1*sigma_2
cov_2_2[2, 0] = rho_1_3*sigma_1*sigma_3
cov_2_2[2, 1] = rho_2_3*sigma_2*sigma_3
cov_2_2[0, 1] = rho_1_2*sigma_1*sigma_2
cov_2_2[0, 2] = rho_1_3*sigma_1*sigma_3
cov_2_2[1, 2] = rho_2_3*sigma_2*sigma_3
                         
sigma_1 = 8
sigma_2 = 8
sigma_3 = 8
cov_2_3 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_2_3[1, 0] = rho_1_2*sigma_1*sigma_2
cov_2_3[2, 0] = rho_1_3*sigma_1*sigma_3
cov_2_3[2, 1] = rho_2_3*sigma_2*sigma_3
cov_2_3[0, 1] = rho_1_2*sigma_1*sigma_2
cov_2_3[0, 2] = rho_1_3*sigma_1*sigma_3
cov_2_3[1, 2] = rho_2_3*sigma_2*sigma_3
                         
sigma_1 = 9
sigma_2 = 9
sigma_3 = 9
cov_2_4 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_2_4[1, 0] = rho_1_2*sigma_1*sigma_2
cov_2_4[2, 0] = rho_1_3*sigma_1*sigma_3
cov_2_4[2, 1] = rho_2_3*sigma_2*sigma_3
cov_2_4[0, 1] = rho_1_2*sigma_1*sigma_2
cov_2_4[0, 2] = rho_1_3*sigma_1*sigma_3
cov_2_4[1, 2] = rho_2_3*sigma_2*sigma_3
                         
sigma_1 = 10
sigma_2 = 10
sigma_3 = 10
cov_2_5 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_2_5[1, 0] = rho_1_2*sigma_1*sigma_2
cov_2_5[2, 0] = rho_1_3*sigma_1*sigma_3
cov_2_5[2, 1] = rho_2_3*sigma_2*sigma_3
cov_2_5[0, 1] = rho_1_2*sigma_1*sigma_2
cov_2_5[0, 2] = rho_1_3*sigma_1*sigma_3
cov_2_5[1, 2] = rho_2_3*sigma_2*sigma_3

In [ ]:
mean_1 = torch.stack((mean_1_1, mean_1_2, mean_1_3, mean_1_4, mean_1_5), dim=0)

In [ ]:
mean_2 = torch.stack((mean_2_1, mean_2_2, mean_2_3, mean_2_4, mean_2_5), dim=0)

In [ ]:
mean = torch.stack((mean_1, mean_2), dim=0)

In [ ]:
cov_1 = torch.stack((cov_1_1, cov_1_2, cov_1_3, cov_1_4, cov_1_5), dim=0)
cov_2 = torch.stack((cov_2_1, cov_2_2, cov_2_3, cov_2_4, cov_2_5), dim=0)

In [ ]:
cov = torch.stack((cov_1, cov_2), dim=0)

In [ ]:
cov.shape

In [ ]:
mean.shape

In [ ]:
normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(mean, covariance_matrix=cov)

In [ ]:
y = torch.Tensor([[1,1,1], [2,2,2], [3,3,3], [4,4,4], [5,5,5]])

In [ ]:
y.shape

In [ ]:
y = torch.stack((y, y), dim=0)

In [ ]:
y.shape

In [ ]:
torch.exp(normal_3d.log_prob(y))

In [ ]:
r.shape

In [ ]:
r

In [ ]:
normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(mean_1, covariance_matrix=cov_1)

In [ ]:
y = torch.Tensor([[1,1,1], [2,2,2], [3,3,3], [4,4,4], [5,5,5]])

In [ ]:
y.shape

In [ ]:
mean_1 = mean_1.unsqueeze(0)
cov_1 = cov_1.unsqueeze(0)

In [ ]:
mean_1.shape

In [ ]:
cov_1.shape

In [ ]:
normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(mean_1, covariance_matrix=cov_1)

In [ ]:
y = torch.Tensor([[1,1,1], [2,2,2], [3,3,3], [4,4,4], [5,5,5]])

In [ ]:
y.shape

In [ ]:
y = y.unsqueeze(0)

In [ ]:
y.shape

In [ ]:
torch.exp(normal_3d.log_prob(y))

In [ ]:
y = torch.Tensor([[1,2,3]])

In [ ]:
y = torch.Tensor([[1, 1, 1],[1, 2, 3]])

In [ ]:
y = y.unsqueeze(1)

In [ ]:
y.shape

In [ ]:
y = y.unsqueeze(1)

In [ ]:
y.shape

In [ ]:
r = torch.exp(normal_3d.log_prob(y))

In [ ]:
r.shape

In [ ]:
r

In [ ]:
mean.shape

In [ ]:
cov = torch.stack((cov, cov_1), dim=0)

In [ ]:
cov.shape

In [ ]:
cov[0]

In [ ]:
cov[1]

In [ ]:
normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(mean, covariance_matrix=cov)

In [ ]:
y = torch.Tensor([[1, 1, 1],[1, 2, 3]])

In [ ]:
y = y.unsqueeze(1)

In [ ]:
y.shape

In [ ]:
torch.exp(normal_3d.log_prob(y))

In [ ]:
y = torch.Tensor([[1,1,1]])

In [ ]:
y.shape

In [ ]:
torch.exp(normal_3d.log_prob(y))

In [ ]:
y.shape

In [ ]:
mean = torch.zeros(3)
sigma_1 = 4
sigma_2 = 4
sigma_3 = 4
cov = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov[1, 0] = rho_1_2*sigma_1*sigma_2
cov[2, 0] = rho_1_3*sigma_1*sigma_3
cov[2, 1] = rho_2_3*sigma_2*sigma_3
cov[0, 1] = rho_1_2*sigma_1*sigma_2
cov[0, 2] = rho_1_3*sigma_1*sigma_3
cov[1, 2] = rho_2_3*sigma_2*sigma_3
normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(mean, covariance_matrix=cov)

y = torch.Tensor([[1, 2, 3]])
torch.exp(normal_3d.log_prob(y))

In [ ]:
mean = torch.zeros(3)
sigma_1 = 2
sigma_2 = 2
sigma_3 = 2
cov_1 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_1[1, 0] = rho_1_2*sigma_1*sigma_2
cov_1[2, 0] = rho_1_3*sigma_1*sigma_3
cov_1[2, 1] = rho_2_3*sigma_2*sigma_3
cov_1[0, 1] = rho_1_2*sigma_1*sigma_2
cov_1[0, 2] = rho_1_3*sigma_1*sigma_3
cov_1[1, 2] = rho_2_3*sigma_2*sigma_3
normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(mean, covariance_matrix=cov_1)

y = torch.Tensor([[1, 2, 3]])
torch.exp(normal_3d.log_prob(y))

In [ ]:
import numpy as np
from mayavi import mlab

s = np.zeros([21, 21, 21])
for i in range(21):
    for j in range(21):
        for k in range(21):
            s[i, j, k] = torch.exp(normal_3d.log_prob(torch.FloatTensor([i-10, j-10, k-10])))
            
src = mlab.pipeline.scalar_field(s)
mlab.pipeline.iso_surface(src, contours=[s.min()+0.1*s.ptp(), ], opacity=0.3)
mlab.pipeline.iso_surface(src, contours=[s.max()-0.1*s.ptp(), ],)

In [ ]:
x = np.linspace(0, 10, 1001)
y = np.linspace(0, 10, 1001)
z = np.linspace(0, 10, 1001)
x_p = x + 0.5 * np.random.randn(1001)
y_p = y + 0.5 * np.random.randn(1001)
z_p = z + 0.5 * np.random.randn(1001)

In [ ]:
ax = plt.axes(projection='3d')
ax.scatter(x, y, z)
ax.scatter(x_p, y_p, z_p)

In [ ]:
class MDN(nn.Module):
    def __init__(self, n_input, n_hidden, n_gaussians):
        super(MDN, self).__init__()
        self.l_h = nn.Sequential(
            nn.Linear(n_input, n_hidden),
            nn.Tanh()
        )
        self.l_pi = nn.Linear(n_hidden, n_gaussians)
        
        self.l_mu_x = nn.Linear(n_hidden, n_gaussians)
        self.l_sigma_x = nn.Linear(n_hidden, n_gaussians)
        
        self.l_mu_y = nn.Linear(n_hidden, n_gaussians)
        self.l_sigma_y = nn.Linear(n_hidden, n_gaussians)
        
        self.l_mu_z = nn.Linear(n_hidden, n_gaussians)
        self.l_sigma_z = nn.Linear(n_hidden, n_gaussians)
        
        # self.l_correlation_x_y = nn.Linear(n_hidden, n_gaussians)
        # self.l_correlation_x_z = nn.Linear(n_hidden, n_gaussians)
        # self.l_correlation_y_z = nn.Linear(n_hidden, n_gaussians)
        
        self.l_correlation_x_y = nn.Sequential(
            nn.Linear(n_hidden, n_gaussians),
            nn.Tanh()
        )
        self.l_correlation_x_z = nn.Sequential(
            nn.Linear(n_hidden, n_gaussians),
            nn.Tanh()
        )
        self.l_correlation_y_z = nn.Sequential(
            nn.Linear(n_hidden, n_gaussians),
            nn.Tanh()
        )
    
    def forward(self, x):
        h = self.l_h(x)
        # print("h", h.shape)
        # print("h[0]", h[0, :])
        pi = F.softmax(self.l_pi(h), -1)
        # print("pi", pi.shape)
        # print("pi[0]", pi[0, :])
        mu_x = self.l_mu_x(h)
        # print("mu_x", pi.shape)
        mu_y = self.l_mu_y(h)
        # print("mu_y", pi.shape)
        mu_z = self.l_mu_z(h)
        # print("mu_z", pi.shape)
        
        # use exp to ensure positive range
        sigma_x = torch.exp(self.l_sigma_x(h))
        # print("sigma_x", pi.shape)
        sigma_y = torch.exp(self.l_sigma_y(h))
        # print("sigma_y", pi.shape)
        sigma_z = torch.exp(self.l_sigma_z(h))
        # print("sigma_z", pi.shape)
        
        # use tanh to ensoure range of (-1, 1)
        correlation_x_y = self.l_correlation_x_y(h)
        # print("correlation_x_y", pi.shape)
        # print("correlation_x_y[0]", correlation_x_y[0, :])
        correlation_x_z = self.l_correlation_x_z(h)
        # print("correlation_x_z", pi.shape)
        # print("correlation_x_z[0]", correlation_x_z[0, :])
        correlation_y_z = self.l_correlation_y_z(h)
        # print("correlation_y_z", pi.shape)
        # print("correlation_y_z[0]", correlation_y_z[0, :])
        
        return pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z, correlation_x_y, correlation_x_z, correlation_y_z

model = MDN(3, n_hidden=20, n_gaussians=5)
#if torch.cuda.is_available():
#    model.cuda()

optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def mdn_loss_fn(y, pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z, correlation_x_y, correlation_x_z, correlation_y_z):
    # print("mu_x shape", mu_x.shape)
    # mu = torch.stack((mu_x, mu_y, mu_z), 2)
    # print("mu shape ", mu.shape)
    # print("mu[0]", mu[0])
    
    ##############################################################################
    
    size = y.shape[0]
    n_gaussians = pi.shape[1]
    # print("sample size: ", size)
    # print("num of gaus: ", n_gaussians)
    # print("correlation size: ", correlation_x_y.shape)
    # build mean matrix
    mean = torch.zeros([size, n_gaussians, 3])
    # build covariance matrix with standard trivariate normal distribution
    cov = torch.ones([size, n_gaussians, 3, 3])
    cov[:, :, 0, 1] = correlation_x_y
    cov[:, :, 1, 0] = correlation_x_y
    cov[:, :, 0, 2] = correlation_x_z
    cov[:, :, 2, 0] = correlation_x_z
    cov[:, :, 1, 2] = correlation_y_z
    cov[:, :, 2, 1] = correlation_y_z
    
    print("mean: ", mean.shape)
    print("cov: ", cov.shape)
    
    print("mean is cuda: ", mean.is_cuda)
    print(" cov is cuda: ", cov.is_cuda)
    
    
    
    
    '''
    w = y[0]**2*(correlation_y_z**2 - 1) +\
        y[1]**2*(correlation_x_z**2 - 1) +\
        y[2]**2*(correlation_x_y**2 - 1) +\
        2*(y[0]*y[:1]*(correlation_x_y - correlation_x_z*correlation_y_z) +\
           y[0]*y[:2]*(correlation_x_z - correlation_x_y*correlation_y_z) +\
           y[1]*y[:2]*(correlation_y_z - correlation_x_y*correlation_x_z))
    
    return w
    '''
    '''
    cov = torch.Tensor([[sigma_x**2, 0, 0], [0, sigma_y**2, 0], [0, 0, sigma_z**2]])
    cov[1, 0] = correlation_x_y*sigma_x*sigma_y
    cov[2, 0] = correlation_x_z*sigma_x*sigma_z
    cov[2, 1] = correlation_y_z*sigma_y*sigma_z
    cov[0, 1] = correlation_x_y*sigma_x*sigma_y
    cov[0, 2] = correlation_x_z*sigma_x*sigma_z
    cov[1, 2] = correlation_y_z*sigma_y*sigma_z
    '''
    m = torch.distributions.multivariate_normal.MultivariateNormal(loc=mean, covariance_matrix=cov)
    # normalize input y
    y_repeat = y.unsqueeze(1).repeat(1, n_gaussians, 1)
    # print(y_repeat.shape)
    # print(y_repeat[0])
    mu_all = torch.stack((mu_x, mu_y, mu_z), dim = 2)
    print("mu_all shape: ", mu_all.shape)
    y_sub = y_repeat - mu_all
    print("y_sub shape", y_sub.shape)
    sigma_all = torch.stack((sigma_x, sigma_y, sigma_z), dim = 2)
    print("sigma", sigma_x.shape)
    print("sigma all", sigma_all.shape)
    
    y_normal = torch.div(y_sub, sigma_all)
    
    print("y_normal size", y_normal.shape)
    
    # y: [x, y, z]; row: x, y, z; column: N, number of samples
    # y: N x 3
    # loss: N x 1
    print(y_normal.is_cuda)
    loss = torch.exp(m.log_prob(y_normal))
    print("loss shape: ", loss.shape)
    print("pi shape: ", pi.shape)
    loss = torch.sum(loss * pi, dim=1)
    loss = -torch.log(loss)
    return torch.mean(loss)

In [ ]:
######################################################333

In [ ]:
for epoch in range(1):
    pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z, correlation_x_y, correlation_x_z, correlation_y_z = model(pd_label_tensor)
    loss = mdn_loss_fn(gt_label_tensor, pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z, correlation_x_y, correlation_x_z, correlation_y_z)
    # optimizer.zero_grad()
    # loss.backward()
    # optimizer.step()
    # if epoch % 1000 == 0:
    #     print(loss.data.tolist())

In [ ]:
mean.shape

In [ ]:
t = torch.rand(2,3)
print(t)
tr = t.repeat(2, 3)
print(tr)

In [ ]:
a = torch.Tensor([[1,2],[4,5],[7,8]])

In [ ]:
b = torch.Tensor([[7,8],[6,4],[1,2]])

In [ ]:
a

In [ ]:
b

In [ ]:
a*b

In [ ]:
torch.sum(a*b, dim=1)

In [ ]:
tt[0,0,0] = 7

In [ ]:
tt

In [ ]:
t

In [ ]:
t[0, 0] = 7

In [ ]:
t

In [ ]:
tr

In [ ]:
correlation_y_z

In [ ]:
0.2939**2

In [ ]:
a

In [ ]:
torch.square(a)

In [ ]:
torch.square(d)

In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(gt_label_tensor[:,0], gt_label_tensor[:,1], gt_label_tensor[:,2], 'gray')
ax.plot3D(pd_label_tensor[:,0], pd_label_tensor[:,1], pd_label_tensor[:,2], 'red')

In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(pd_label_tensor[:,0], pd_label_tensor[:,1], pd_label_tensor[:,2], 'gray')

In [ ]:
plt.plot(gt_label_tensor[:,2])
plt.plot(pd_label_tensor[:,2])

In [ ]:
a

In [ ]:
input_tensor

In [ ]:
test_1_loss = []
for i in range(1, 11):
    window_size = i
    PATH = "model/with_early_stop_after_400/model_w_" + str(i) + ".pt"

    test_set_x_1, test_set_y_1, test_set_z_1 = load_test_data('../../../../performance_test/data/test/test_1.csv')
    test_set_x_2, test_set_y_2, test_set_z_2 = load_test_data('../../../../performance_test/data/test/test_2.csv')
    test_set_x_3, test_set_y_3, test_set_z_3 = load_test_data('../../../../performance_test/data/test/test_3.csv')
    test_set_x_4, test_set_y_4, test_set_z_4 = load_test_data('../../../../performance_test/data/test/test_4.csv')
    test_set_x_5, test_set_y_5, test_set_z_5 = load_test_data('../../../../performance_test/data/test/test_5.csv')

    # do min-max-scaling for each test data set
    show_statistic(test_set_x_1)
    min_max_scaling(test_set_x_1)
    show_statistic(test_set_x_1)
    min_max_scaling(test_set_x_2)
    min_max_scaling(test_set_x_3)
    min_max_scaling(test_set_x_4)
    min_max_scaling(test_set_x_5)

    min_max_scaling(test_set_y_1)
    min_max_scaling(test_set_y_2)
    min_max_scaling(test_set_y_3)
    min_max_scaling(test_set_y_4)
    min_max_scaling(test_set_y_5)

    min_max_scaling(test_set_z_1)
    min_max_scaling(test_set_z_2)
    min_max_scaling(test_set_z_3)
    min_max_scaling(test_set_z_4)
    min_max_scaling(test_set_z_5)


    show_statistic(test_set_x_1)
    # do normalization on x of validation test set
    normalize_one(test_set_x_1, x_mean, x_std)
    show_statistic(test_set_x_1)
    normalize_one(test_set_x_2, x_mean, x_std)
    normalize_one(test_set_x_3, x_mean, x_std)
    normalize_one(test_set_x_4, x_mean, x_std)
    normalize_one(test_set_x_5, x_mean, x_std)
    # do normalization on y of validation test set
    normalize_one(test_set_y_1, y_mean, y_std)
    normalize_one(test_set_y_2, y_mean, y_std)
    normalize_one(test_set_y_3, y_mean, y_std)
    normalize_one(test_set_y_4, y_mean, y_std)
    normalize_one(test_set_y_5, y_mean, y_std)
    # do normalization on z of validation test set
    normalize_one(test_set_z_1, z_mean, z_std)
    normalize_one(test_set_z_2, z_mean, z_std)
    normalize_one(test_set_z_3, z_mean, z_std)
    normalize_one(test_set_z_4, z_mean, z_std)
    normalize_one(test_set_z_5, z_mean, z_std)
    # show_statistic(train_set_x_1)


    test_dataset_1, test_label_1 = construct_test_tensor(test_set_x_1,
                                                         test_set_y_1,
                                                         test_set_z_1,
                                                         window_size)
    test_dataset_2, test_label_2 = construct_test_tensor(test_set_x_2,
                                                         test_set_y_2,
                                                         test_set_z_2,
                                                         window_size)
    test_dataset_3, test_label_3 = construct_test_tensor(test_set_x_3,
                                                         test_set_y_3,
                                                         test_set_z_3,
                                                         window_size)
    test_dataset_4, test_label_4 = construct_test_tensor(test_set_x_4,
                                                         test_set_y_4,
                                                         test_set_z_4,
                                                         window_size)
    test_dataset_5, test_label_5 = construct_test_tensor(test_set_x_5,
                                                         test_set_y_5,
                                                         test_set_z_5,
                                                         window_size)

    test_set_1 = MyTestDataSet(test_dataset_1, test_label_1)
    test_set_2 = MyTestDataSet(test_dataset_2, test_label_2)
    test_set_3 = MyTestDataSet(test_dataset_3, test_label_3)
    test_set_4 = MyTestDataSet(test_dataset_4, test_label_4)
    test_set_5 = MyTestDataSet(test_dataset_5, test_label_5)
    print(len(test_set_1))
    print(len(test_set_2))
    print(len(test_set_3))
    print(len(test_set_4))
    print(len(test_set_5))
    show_statistic(test_set_x_1)
    show_statistic(test_set_x_2)
    show_statistic(test_set_x_3)
    show_statistic(test_set_x_4)
    show_statistic(test_set_x_5)


    batch_size = 650
    test_loader_1 = DataLoader(test_set_1, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_2 = DataLoader(test_set_2, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_3 = DataLoader(test_set_3, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_4 = DataLoader(test_set_4, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_5 = DataLoader(test_set_5, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    
    input_dim = 3
    hidden_dim = 100
    layer_dim = 1
    output_dim = 3

    load_model = my_model.LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
    load_model.load_state_dict(torch.load(PATH))
    load_model.eval()
    # mmodel = torch.load(PATH)

    criterion = nn.MSELoss()

    if torch.cuda.is_available():
        load_model.cuda()

    # get test results
    seq_dim = window_size
    input_dim = 3
    # test_seq = []
    test_predd = []
    # test_gt = []
    total_test_loss = 0.0
    test_batch = 0
    for i, (seqs, labels) in enumerate(test_loader_1):
        if torch.cuda.is_available():
            seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            seqs = Variable(seqs.view(-1, seq_dim, input_dim))

        outputs = load_model(seqs)
        loss = criterion(outputs, labels)
        total_test_loss += loss.data.item()
        test_predd.append(outputs)
        test_batch = i + 1

    print(total_test_loss/test_batch)
    test_1_loss.append(total_test_loss/te    input_dim = 3
    hidden_dim = 100
    layer_dim = 1
    output_dim = 3

    load_model = my_model.LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
    load_model.load_state_dict(torch.load(PATH))
    load_model.eval()
    # mmodel = torch.load(PATH)

    criterion = nn.MSELoss()

    if torch.cuda.is_available():
        load_model.cuda()

    # get test results
    seq_dim = window_size
    input_dim = 3
    # test_seq = []
    test_predd = []
    # test_gt = []
    total_test_loss = 0.0
    test_batch = 0
    for i, (seqs, labels) in enumerate(test_loader_1):
        if torch.cuda.is_available():
            seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            seqs = Variable(seqs.view(-1, seq_dim, input_dim))

        outputs = load_model(seqs)
        loss = criterion(outputs, labels)
        total_test_loss += loss.data.item()
        test_predd.append(outputs)
        test_batch = i + 1

    print(total_test_loss/test_batch)st_batch)

    for i in range(len(test_predd)):
        if (i == 0):
            pred = test_predd[i].cpu().detach().numpy()
        else:
            pred = np.append(pred, test_predd[i].cpu().detach().numpy(), axis = 0)

    from mpl_toolkits import mplot3d
    fig = plt.figure()
    ax = plt.axes(projection='3d')

    # Data for a three-dimensional line
    zline = np.linspace(0, 15, 1000)
    xline = np.sin(zline)
    yline = np.cos(zline)
    ax.plot3D(test_set_x_1, test_set_y_1, test_set_z_1, 'gray')
    ax.plot3D(pred[:,0], pred[:,1], pred[:,2], 'red')

In [ ]:
test_1_loss

In [ ]:
#find window size 6 is optimal

In [ ]:
x = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

x_pos = [i for i, _ in enumerate(x)]

# plt.bar(x_pos, test_1_loss)
plt.bar(x_pos, test_1_loss, color=['tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:green',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue'], zorder = 3)

plt.grid(zorder=0)

plt.xlabel("Input window size", fontsize=14)
plt.ylabel("MSE loss", fontsize=14)
plt.xticks(x_pos, x)

plt.ylim([0.017,0.0255])
# plt.show()

Plot result without normalization

In [ ]:
test_3_loss = []
for i in range(1, 11):
    window_size = i
    PATH = "model_v3/with_early_stop_after_400_without_normalization/model_w_" + str(i) + ".pt"

    test_set_x_1, test_set_y_1, test_set_z_1 = load_test_data('../../../../performance_test/data/test/test_1.csv')
    test_set_x_2, test_set_y_2, test_set_z_2 = load_test_data('../../../../performance_test/data/test/test_2.csv')
    test_set_x_3, test_set_y_3, test_set_z_3 = load_test_data('../../../../performance_test/data/test/test_3.csv')
    test_set_x_4, test_set_y_4, test_set_z_4 = load_test_data('../../../../performance_test/data/test/test_4.csv')
    test_set_x_5, test_set_y_5, test_set_z_5 = load_test_data('../../../../performance_test/data/test/test_5.csv')

    '''
    # do min-max-scaling for each test data set
    show_statistic(test_set_x_1)
    min_max_scaling(test_set_x_1)
    show_statistic(test_set_x_1)
    min_max_scaling(test_set_x_2)
    min_max_scaling(test_set_x_3)
    min_max_scaling(test_set_x_4)
    min_max_scaling(test_set_x_5)

    min_max_scaling(test_set_y_1)
    min_max_scaling(test_set_y_2)
    min_max_scaling(test_set_y_3)
    min_max_scaling(test_set_y_4)
    min_max_scaling(test_set_y_5)

    min_max_scaling(test_set_z_1)
    min_max_scaling(test_set_z_2)
    min_max_scaling(test_set_z_3)
    min_max_scaling(test_set_z_4)
    min_max_scaling(test_set_z_5)
    '''
    '''
    show_statistic(test_set_x_1)
    # do normalization on x of validation test set
    normalize_one(test_set_x_1, x_mean, x_std)
    show_statistic(test_set_x_1)
    normalize_one(test_set_x_2, x_mean, x_std)
    normalize_one(test_set_x_3, x_mean, x_std)
    normalize_one(test_set_x_4, x_mean, x_std)
    normalize_one(test_set_x_5, x_mean, x_std)
    # do normalization on y of validation test set
    normalize_one(test_set_y_1, y_mean, y_std)
    normalize_one(test_set_y_2, y_mean, y_std)
    normalize_one(test_set_y_3, y_mean, y_std)
    normalize_one(test_set_y_4, y_mean, y_std)
    normalize_one(test_set_y_5, y_mean, y_std)
    # do normalization on z of validation test set
    normalize_one(test_set_z_1, z_mean, z_std)
    normalize_one(test_set_z_2, z_mean, z_std)
    normalize_one(test_set_z_3, z_mean, z_std)
    normalize_one(test_set_z_4, z_mean, z_std)
    normalize_one(test_set_z_5, z_mean, z_std)
    # show_statistic(train_set_x_1)
    '''

    test_dataset_1, test_label_1 = construct_test_tensor(test_set_x_1,
                                                         test_set_y_1,
                                                         test_set_z_1,
                                                         window_size)
    test_dataset_2, test_label_2 = construct_test_tensor(test_set_x_2,
                                                         test_set_y_2,
                                                         test_set_z_2,
                                                         window_size)
    test_dataset_3, test_label_3 = construct_test_tensor(test_set_x_3,
                                                         test_set_y_3,
                                                         test_set_z_3,
                                                         window_size)
    test_dataset_4, test_label_4 = construct_test_tensor(test_set_x_4,
                                                         test_set_y_4,
                                                         test_set_z_4,
                                                         window_size)
    test_dataset_5, test_label_5 = construct_test_tensor(test_set_x_5,
                                                         test_set_y_5,
                                                         test_set_z_5,
                                                         window_size)

    test_set_1 = MyTestDataSet(test_dataset_1, test_label_1)
    test_set_2 = MyTestDataSet(test_dataset_2, test_label_2)
    test_set_3 = MyTestDataSet(test_dataset_3, test_label_3)
    test_set_4 = MyTestDataSet(test_dataset_4, test_label_4)
    test_set_5 = MyTestDataSet(test_dataset_5, test_label_5)
    print(len(test_set_1))
    print(len(test_set_2))
    print(len(test_set_3))
    print(len(test_set_4))
    print(len(test_set_5))
    show_statistic(test_set_x_1)
    show_statistic(test_set_x_2)
    show_statistic(test_set_x_3)
    show_statistic(test_set_x_4)
    show_statistic(test_set_x_5)


    batch_size = 650
    test_loader_1 = DataLoader(test_set_1, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_2 = DataLoader(test_set_2, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_3 = DataLoader(test_set_3, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_4 = DataLoader(test_set_4, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_5 = DataLoader(test_set_5, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    
    input_dim = 3
    hidden_dim = 100
    layer_dim = 1
    output_dim = 3

    load_model = my_model.LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
    load_model.load_state_dict(torch.load(PATH))
    load_model.eval()
    # mmodel = torch.load(PATH)

    criterion = nn.MSELoss()

    if torch.cuda.is_available():
        load_model.cuda()

    # get test results
    seq_dim = window_size
    input_dim = 3
    # test_seq = []
    test_predd = []
    # test_gt = []
    total_test_loss = 0.0
    test_batch = 0
    for i, (seqs, labels) in enumerate(test_loader_3):
        if torch.cuda.is_available():
            seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            seqs = Variable(seqs.view(-1, seq_dim, input_dim))

        outputs = load_model(seqs)
        loss = criterion(outputs, labels)
        total_test_loss += loss.data.item()
        test_predd.append(outputs)
        test_batch = i + 1

    print(total_test_loss/test_batch)
    test_3_loss.append(total_test_loss/test_batch)

    for i in range(len(test_predd)):
        if (i == 0):
            pred = test_predd[i].cpu().detach().numpy()
        else:
            pred = np.append(pred, test_predd[i].cpu().detach().numpy(), axis = 0)

    from mpl_toolkits import mplot3d
    fig = plt.figure()
    ax = plt.axes(projection='3d')

    # Data for a three-dimensional line
    zline = np.linspace(0, 15, 1000)
    xline = np.sin(zline)
    yline = np.cos(zline)
    ax.plot3D(test_set_x_3, test_set_y_3, test_set_z_3, 'gray')
    ax.plot3D(pred[:,0], pred[:,1], pred[:,2], 'red')

In [ ]:
test_3_loss

In [ ]:
min(test_3_loss)

In [ ]:
x = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

x_pos = [i for i, _ in enumerate(x)]

# plt.bar(x_pos, test_1_loss)
plt.bar(x_pos, test_3_loss, color=['tab:blue',
                                   'tab:blue',
                                   'tab:green',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue'], zorder = 3)

plt.grid(zorder=0)

plt.xlabel("Input Window Size", fontsize=12)
plt.ylabel("MSE Loss", fontsize=12)
plt.xticks(x_pos, x)

# plt.ylim([0.017,0.0255])
plt.ylim([0.025,0.039])
# plt.show()